### 패키지 설치

In [ ]:
!pip install langchain langchain-core langchain-community langchain-google-genai

### 모듈 임포트 및 구글 API 키 설정

In [ ]:
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import create_agent
from langchain.tools import tool

# API 키 설정
os.environ["GOOGLE_API_KEY"] = "your-api-key"

### 집안 상태 시뮬레이션을 위한 상태 변수 정의

In [ ]:
# ============================================================
# 스마트홈 기기 제어 도구 정의
# ============================================================

# 현재 집안 상태 (시뮬레이션용)
home_state = {
    "lights": {"living_room": {"on": False, "brightness": 0, "color": "white"}},
    "temperature": {"current": 19, "target": 22, "mode": "off"},
    "curtains": {"living_room": "open"},
    "music": {"playing": False, "volume": 0},
    "tv": {"on": False, "channel": 1}
}

### 스마트홈 기기 제어를 위한 도구 함수 정의

1. control_temperature: 설정 온도를 변경할 때 사용.
2. get_temperature: 현재 온도를 확인하기 위해 사용.
3. control_curtains: 커튼을 제어하기 위해 사용.
4. control_music: CD플레이어와 스피커를 제어하는데 사용.
5. control_lights: 조명을 켜거나 끄고, 밝기를 조절할 때 사용.
6. control_tv: TV를 켜거나 특정 앱을 실행할 때 사용.
7. get_home_status: 집안의 모든 기기 상태를 확인할 때 사용.

In [ ]:
@tool
def control_lights(room: str, action: str, brightness: int = 50, color: str = "white") -> str:
    """
    조명을 제어합니다.

    Args:
        room: 방 이름 (예: living_room, bedroom)
        action: 'on' 또는 'off'
        brightness: 밝기 (0-100), action이 'on'일 때 사용
        color: 색상 (white, warm, cool, red, blue 등), action이 'on'일 때 사용

    Returns:
        조명 제어 결과 메시지
    """
    if room not in home_state["lights"]:
        home_state["lights"][room] = {"on": False, "brightness": 0, "color": "white"}

    if action == "on":
        home_state["lights"][room] = {"on": True, "brightness": brightness, "color": color}
        return f"{room} 조명을 켰습니다. 밝기: {brightness}%, 색상: {color}"
    elif action == "off":
        home_state["lights"][room]["on"] = False
        return f"{room} 조명을 껐습니다."
    else:
        return "action은 'on' 또는 'off'여야 합니다."

@tool
def control_temperature(action: str, target_temp: int = 22) -> str:
    """
    온도를 제어합니다 (에어컨/히터).

    Args:
        action: 'cool' (냉방), 'heat' (난방), 'off' (끄기)
        target_temp: 목표 온도 (°C)

    Returns:
        온도 제어 결과 메시지
    """
    current = home_state["temperature"]["current"]

    if action == "off":
        home_state["temperature"]["mode"] = "off"
        return f"냉난방을 껐습니다. 현재 온도: {current}°C"
    elif action in ["cool", "heat"]:
        home_state["temperature"]["mode"] = action
        home_state["temperature"]["target"] = target_temp
        return f"{action} 모드로 설정했습니다. 목표 온도: {target_temp}°C, 현재 온도: {current}°C"
    else:
        return "action은 'cool', 'heat', 또는 'off'여야 합니다."

@tool
def get_temperature() -> str:
    """
    현재 실내 온도를 확인합니다.

    Returns:
        현재 온도 정보
    """
    temp = home_state["temperature"]["current"]
    mode = home_state["temperature"]["mode"]
    target = home_state["temperature"]["target"]

    if mode == "off":
        return f"현재 온도: {temp}°C (냉난방 꺼짐)"
    else:
        return f"현재 온도: {temp}°C, 목표: {target}°C, 모드: {mode}"

@tool
def control_curtains(room: str, action: str) -> str:
    """
    커튼을 제어합니다.

    Args:
        room: 방 이름 (예: living_room, bedroom)
        action: 'open' (열기) 또는 'close' (닫기)

    Returns:
        커튼 제어 결과 메시지
    """
    if room not in home_state["curtains"]:
        home_state["curtains"][room] = "open"

    if action in ["open", "close"]:
        home_state["curtains"][room] = action
        return f"{room} 커튼을 {action}했습니다."
    else:
        return "action은 'open' 또는 'close'여야 합니다."

@tool
def control_music(action: str, volume: int = 30) -> str:
    """
    CD플레이어/스피커를 제어합니다.

    Args:
        action: 'play' (재생) 또는 'stop' (정지)
        volume: 볼륨 (0-100)

    Returns:
        CD플레이어 제어 결과 메시지
    """
    if action == "play":
        home_state["music"]["playing"] = True
        home_state["music"]["volume"] = volume
        return f"CD플레이어를 재생합니다. 볼륨: {volume}%"
    elif action == "stop":
        home_state["music"]["playing"] = False
        return "CD플레이어를 정지했습니다."
    else:
        return "action은 'play' 또는 'stop'이어야 합니다."

@tool
def control_tv(action: str, channel: int = 1) -> str:
    """
    TV를 제어합니다.

    Args:
        action: 'on' (켜기) 또는 'off' (끄기)
        channel: 채널 번호

    Returns:
        TV 제어 결과 메시지
    """
    if action == "on":
        home_state["tv"]["on"] = True
        home_state["tv"]["channel"] = channel
        return f"TV를 켰습니다. 채널: {channel}"
    elif action == "off":
        home_state["tv"]["on"] = False
        return "TV를 껐습니다."
    else:
        return "action은 'on' 또는 'off'여야 합니다."

@tool
def get_home_status() -> str:
    """
    현재 집안의 모든 기기 상태를 확인합니다.

    Returns:
        모든 기기의 현재 상태
    """
    status = "=== 현재 집안 상태 ===\n"

    status += "\n[조명]\n"
    for room, light in home_state["lights"].items():
        status += f"  {room}: {'켜짐' if light['on'] else '꺼짐'}"
        if light['on']:
            status += f" (밝기: {light['brightness']}%, 색상: {light['color']})"
        status += "\n"

    status += f"\n[온도]\n  {home_state["temperature"]["current"]}\n"

    status += "\n[커튼]\n"
    for room, curtain in home_state["curtains"].items():
        status += f"  {room}: {curtain}\n"

    music = home_state["music"]
    status += f"\n[음악]\n  {'재생 중' if music['playing'] else '정지'}"
    if music['playing']:
        status += f" (볼륨: {music['volume']}%)"
    status += "\n"

    tv = home_state["tv"]
    status += f"\n[TV]\n  {'켜짐' if tv['on'] else '꺼짐'}"
    if tv['on']:
        status += f" (채널: {tv['channel']})"
    status += "\n"

    return status

### 위에서 정의한 도구 함수들의 리스트 (에이전트에게 건내줄 예정)

In [ ]:
# 도구 목록
tools = [control_temperature, get_temperature, control_curtains, control_music, control_lights, control_tv, get_home_status]


### 시스템 프롬프트 정의

(경험상 TIP) 항상 문제가 생긴다고 할 수는 없지만 ReAct 프롬프트 방식대로 명시적으로 흔히하는 대로 다음과 같이

```
> Question: 해결해야 할 입력 질문
> Thought: 무엇을 해야 할지...
> Action: 수행할 작업...
> Observation: 작업의 결과
> Final Answer: ...
```

형식 지시를 프롬프트에 포함시키면 agent가 이상하게 행동함 (가령, 강한 환각 현상을 보이는 것처럼 출력 메시지 상으로는 적절한 도구 함수를 고르고 실행한 것처럼 보이는데 실제로 실행하지 않음)

이는 <code>create_agent</code>가 또 내부적으로 ReAct 형식을 자동으로 추가하기 때문에 ReAct 프롬프트가 중복되어 나타나는 문제로 추정됨. 



In [ ]:
# ============================================================
# LLM 및 에이전트 설정
# ============================================================

# LLM 설정
llm = ChatGoogleGenerativeAI(
    model="gemini-flash-lite-latest",
    temperature=0
)

# 시스템 프롬프트 - 재귀적 추론을 유도
system_prompt = """
당신은 스마트 홈을 제어하는 AI 비서입니다. 사용자의 요청을 가장 잘 수행하기 위해 오직 아래의 도구들만을 사용할 수 있습니다:

1. control_temperature: 설정 온도를 변경할 때 사용.
2. get_temperature: 현재 온도를 확인하기 위해 사용.
3. control_curtains: 커튼을 제어하기 위해 사용.
4. control_music: CD플레이어와 스피커를 제어하는데 사용.
5. control_lights: 조명을 켜거나 끄고, 밝기를 조절할 때 사용.
6. control_tv: TV를 켜거나 특정 앱을 실행할 때 사용.
7. get_home_status: 집안의 모든 기기 상태를 확인할 때 사용.

사용자의 질문/명령에 따라 스스로 수행할 도구를 생각해 호출(행동)하세요.
생각/행동/관찰(도구 실행 결과)는 여러번 반복할 수 있습니다.
"""

# 에이전트 생성 (LangChain v1 방식)
agent = create_agent(
    llm,
    tools,
    system_prompt=system_prompt
)

### 실행 예제

In [ ]:
# ============================================================
# 실행 예제
# ============================================================
print("=" * 60)
print("스마트홈 AI 에이전트 - 재귀적 추론 예제")
print("=" * 60)

# 예제 1: 조건부 연쇄 작업 - 영화 보기 모드
print("\n\n[예제 1] 영화 볼 준비해줘")
print("-" * 60)
result1 = agent.invoke({
    "messages": [("user", "영화 볼 준비해줘. 조명은 어둡게, 커튼 닫고, 지금 좀 추우니까, 온도 확인해서 필요하면 조절하고, TV 켜줘.")]
})
print("\n최종 결과:", result1["messages"][-1].content)

In [ ]:
# 예제 2: 상태 확인 후 조건부 실행
print("\n\n[예제 2] 잘 준비해줘")
print("-" * 60)
result2 = agent.invoke({
    "messages": [("user", "아침이야. 출근할 준비를 해줘.")]
})
print("\n최종 결과:", result2["messages"][-1].content)

In [ ]:
# 예제 3: 현재 상태 확인
print("\n\n[예제 3] 집안 상태 확인")
print("-" * 60)
result3 = agent.invoke({
    "messages": [("user", "지금 집안 상태 알려줘")]
})
print("\n최종 결과:", result3["messages"][-1].content)

In [ ]:
# 집안 상태 변수도 확인해보기
home_state

In [ ]:
result4 = agent.invoke({
    "messages": [("user", "지금 집안 온도 알려줘")]
})
print("\n최종 결과:", result4["messages"][-1].content)